# Merge tiffs

Merge the tiffs of cropped over the channel area / field site radar lines into one tiff.

In [1]:
import numpy as np
import rasterio 
import rasterio.merge
import os
import sys
import fiona

#working_dir = '/Volumes/arc_02/REMOTE_SENSING/'
working_dir = "/Users/home/whitefar/DATA/FIELD_ANT_19/2m_REMA_mosaic_DEM/polygons_of_tiffs"


sys.path.append(os.path.abspath(working_dir))
sys.path.append(os.path.abspath('/Users/home/whitefar/DATA/code/'))
os.chdir(working_dir)

In [2]:
polygons_path = "/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/cropped_stripes/"
cropped_REMA_stripes_path = "/Volumes/arc_02/REMOTE_SENSING/REMA/REMA_stripes_1920_fieldsite/"

output_path = "/Users/home/whitefar/DATA/FIELD_ANT_19/2m_REMA_mosaic_DEM/"

selected_stripes_fnames = [f for f in os.listdir(cropped_REMA_stripes_path) if os.path.isfile(os.path.join(cropped_REMA_stripes_path, f))]
selected_stripes_fnames


['SETSM_WV03_20161220_1040010026391800_1040010026480500_seg1_2m_v1.0_dem.tif',
 'SETSM_WV02_20161220_1030010061866000_103001006007BA00_seg1_2m_v1.0_dem.tif',
 'SETSM_WV01_20161027_1020010056BA2E00_1020010058DFD800_seg1_2m_v1.0_dem.tif',
 'SETSM_WV01_20161109_1020010058134D00_10200100576C9100_seg1_2m_v1.0_dem.tif']

In [3]:
# selected_stripes_polygon = "/Users/home/whitefar/DATA/FIELD_ANT_19/2m_REMA_mosaic_DEM/polygons_of_tiffs/polygons_for_merge.gpkg"
# with fiona.open(selected_stripes_polygon, "r") as shapefile:
#     selected_stripe_names = [feature["properties"]["name"] for feature in shapefile]
# selected_stripes_fnames = [names + "_dem.tif"  for names in selected_stripe_names]
# selected_stripes_fnames

In [6]:
from contextlib import ExitStack

with ExitStack() as stack:
    files = [stack.enter_context(rasterio.open(cropped_REMA_stripes_path + fname)) for fname in selected_stripes_fnames]
    # All opened files will automatically be closed at the end of
    # the with statement, even if attempts to open files later
    # in the list raise an exception




# with rasterio.open(cropped_REMA_stripes_path + selected_stripes_fnames[0],"r") as src0,\
#      rasterio.open(cropped_REMA_stripes_path + selected_stripes_fnames[1],"r") as src1,\
#      rasterio.open(cropped_REMA_stripes_path + selected_stripes_fnames[2],"r") as src2,\
#      rasterio.open(cropped_REMA_stripes_path + selected_stripes_fnames[3],"r") as src3:
    
    print("1/4")
    
    out_image, out_transform =  rasterio.merge.merge(datasets=files,
                                                     bounds=None,
                                                     res=None,
                                                     nodata=None,
                                                     precision=7,
                                                     indexes=None)
    print("2/4")
    
    out_meta = files[0].meta.copy()
    data_type = out_image.dtype
        
    print("3/4")
    
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform,
                     "dtype" : data_type
                     }
                    )

    out_name = "2m_REMA_mosaic_DEM"
    
    print("4/4")

with rasterio.open(output_path + out_name + "_dem.tif", "w", **out_meta) as dest:
    dest.write(out_image)

print("done")
    

1/4
2/4
3/4
4/4
done
